# Exploration of Linking Phenoscape and iDigBio



In [34]:
from pyspark.sql.functions import col, lower, split, sum
from pyspark.sql.types import IntegerType

## Linking based on specimen identifiers in Phenoscape

Phenoscape's first project included the identifiers of specimens taken from publications with the phenotypes collected. Some limitations:

1. Only the catalog number and institution code are entered
1. The specimens identified were mentioned in the paper with the phenotypes, there is not a 1:1 correspondence between the measured character and the specific specimen

### Downloading data from Phenoscape

The SPARQL interface (http://yasgui.org/ with the endpoint http://db.phenoscape.org/bigsparql) was used to download a CSV with all distinct specimen identifiers. The query was provided by Jim Balhof:

```
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX ps: <http://purl.org/phenoscape/vocab.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX BFO: <http://purl.obolibrary.org/obo/BFO_>
PREFIX fin: <http://purl.obolibrary.org/obo/UBERON_0008897>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX hint: <http://www.bigdata.com/queryHints#> 
PREFIX pedal_digit_1: <http://purl.obolibrary.org/obo/UBERON_0003631>
PREFIX pectoral_fin: <http://purl.obolibrary.org/obo/UBERON_0000151>
PREFIX prehallux: <http://purl.obolibrary.org/obo/UBERON_0012136>
PREFIX Anura: <http://purl.obolibrary.org/obo/VTO_0002210>
PREFIX length: <http://purl.obolibrary.org/obo/PATO_0000122>
PREFIX dwc: <http://rs.tdwg.org/dwc/terms/>
PREFIX has_external_reference: <http://purl.obolibrary.org/obo/CDAO_0000164>
SELECT DISTINCT ?matrix_taxon ?vto ?vto_label ?collection ?catNum WHERE {
?otu dwc:individualID ?specimen .
?otu rdfs:label ?matrix_taxon .
?otu has_external_reference: ?vto .
?vto rdfs:label ?vto_label .
?specimen dwc:collectionID/rdfs:label ?collection .
?specimen dwc:catalogNumber ?catNum .
}
```

The downloaded CSV had spaces around the delimiter that needed to be trimmed away to have it import nicely into Spark. See `phenoscape_dl_fixup.sh`.

### Characterizing the downloaded data

In [2]:
# Load CSV after placing it on HDFS
ps_spec = (sqlContext
           .read
           .option("header", "true")
           .csv("/home/mjcollin/queryResults_phenoscape_specimens.csv")
           .cache()
           )

In [3]:
print(ps_spec.count())
ps_spec.printSchema()
ps_spec.show(3)

12283
root
 |-- matrix_taxon: string (nullable = true)
 |-- vto: string (nullable = true)
 |-- vto_label: string (nullable = true)
 |-- collection: string (nullable = true)
 |-- catNum: string (nullable = true)

+--------------------+--------------------+--------------------+----------+------+
|        matrix_taxon|                 vto|           vto_label|collection|catNum|
+--------------------+--------------------+--------------------+----------+------+
|Scyliorhinus retifer|http://purl.oboli...|Scyliorhinus retifer|      AMNH| 36777|
|     Mustelus laevis|http://purl.oboli...|   Mustelus mustelus|      AMNH|  4138|
|   Channallabes apus|http://purl.oboli...|   Channallabes apus|      AMNH|  6631|
+--------------------+--------------------+--------------------+----------+------+
only showing top 3 rows



First, lets see what the quality of the catalog numbers are by looking for things that have remarks, text, and other non-catalog number type information stored in the field.

In [4]:
poor_cat_nums = (ps_spec
                 .filter(~ col("catNum").rlike("[0-9]")
                        | col("catNum").like("%(%"))
                 .groupBy(col("catNum"))
                 .count()
                 .orderBy(col("count"), ascending=False)
                 )

print(poor_cat_nums.count())
poor_cat_nums.show(25, truncate=False)
                 

115
+-------------------------------------------------------------------+-----+
|catNum                                                             |count|
+-------------------------------------------------------------------+-----+
|uncat.                                                             |210  |
|V-s/n                                                              |36   |
|uncatalogued                                                       |23   |
|não catalogado                                                     |20   |
|não catalog.                                                       |10   |
|Uncat.                                                             |7    |
|F uncat.                                                           |5    |
|nåo catalog.                                                       |4    |
|não catalog                                                        |4    |
|uncat                                                              |4    |
|NRA    

In [5]:
(poor_cat_nums
 .select(sum(col("count").cast(IntegerType())))
 .show()
)

+-----------------------+
|sum(CAST(count AS INT))|
+-----------------------+
|                    439|
+-----------------------+



It looks like we should set aside 439 out of the 12283 specimen ids as being not valid or requireing interpretation.

What collections are represented?

In [6]:
ps_collections = (ps_spec
                  .groupBy(col("collection"))
                  .count()
                  .orderBy(col("count"), ascending=False)
)
print(ps_collections.count())
ps_collections.show(10)

128
+----------+-----+
|collection|count|
+----------+-----+
|      USNM| 1584|
|      AMNH| 1116|
|     MZUSP|  950|
|      UMMZ|  834|
|      BMNH|  626|
|      FMNH|  583|
|       UAM|  548|
|        KU|  546|
|       OSM|  491|
|      HUMZ|  396|
+----------+-----+
only showing top 10 rows



Which of these are in iDigBio? Here's where we'll need to load up iDigBio to take a look.

In [7]:
idb_df = sqlContext.read.parquet("/guoda/data/idigbio-20171209T023310.parquet")

In [8]:
print(idb_df.count())

106202428


In [17]:
ps_collections_list = [i.collection.lower() for i in ps_collections.collect()]
print(ps_collections_list[0:10])

['usnm', 'amnh', 'mzusp', 'ummz', 'bmnh', 'fmnh', 'uam', 'ku', 'osm', 'humz']


In [16]:
idb_institutions = (idb_df
                   .filter(col("institutioncode").isin(ps_collections_list))
                   .groupBy(col("institutioncode"))
                   .count()
                   .orderBy(col("count"), ascending=False)
                   )
print(idb_institutions.count())
idb_institutions.show(10)

69
+---------------+-------+
|institutioncode|  count|
+---------------+-------+
|           mnhn|7116859|
|           usnm|5020984|
|             ku|2690964|
|            cas|2417139|
|           fmnh|1854607|
|            mcz|1844567|
|             uf|1699668|
|            ypm|1395137|
|           amnh|1140963|
|             cm| 981091|
+---------------+-------+
only showing top 10 rows



Only 69 of the 128 collection codes are not in the institution field. Are they in the collection code field?

In [18]:
idb_collections = (idb_df
                   .filter(col("collectioncode").isin(ps_collections_list))
                   .groupBy(col("collectioncode"))
                   .count()
                   .orderBy(col("count"), ascending=False)
                   )
print(idb_collections.count())
idb_collections.show(10)

49
+--------------+------+
|collectioncode| count|
+--------------+------+
|            pc|461954|
|            uf|347466|
|          uaic|188772|
|          kuvp| 82676|
|            iu| 71561|
|          uamz| 33936|
|            os|  5114|
|            ku|  2161|
|          lacm|  2152|
|          tcwc|  2024|
+--------------+------+
only showing top 10 rows



Yup, a bunch match here too. We'll have to be more generous in the interpretation of the field which will likely lead to over/false matches, especially since there look to be a number of 2-letter codes which are unlikely to be unambigious.

Perhaps there are collections are not in iDigBio? How many specimens from Phenoscape don't have a collection code in iDigBio in either the institution or collection field?

In [47]:
idb_institution_set = set([i.institutioncode for i in idb_institutions.collect()])
idb_collection_set = set([i.collectioncode for i in idb_collections.collect()])
missing_collections = (set(ps_collections_list) - idb_institution_set) - idb_collection_set
print(missing_collections)

{'osm', 'ihb', 'saiab', 'scnu', 'frsku', 'fml', 'icnmhn', 'zsi', 'arc', 'mhnls', 'bsku', 'mbucv', 'nwafc', 'ams', 'cmk', 'mapa', 'gvf', 'faku', 'mac-pay', 'mzb', 'furg', 'zumt', 'ntm', 'msnvr', 'ncip', 'mcng', 'amg', 'nhrm', 'vims', 'cas-iu', 'mzuabcs', 'mepn', 'kfrs', 'izua', 'su', 'fsfrl', 'kiz', 'humz', 'fau', 'huj', 'ibrp', 'ummp', 'adr', 'mcp', 'nlu', 'lgp', 'mm', 'cas-su', 'smwu', 'dbav.uerj', 'mhnm', 'kumf', 'lbuch'}


In [52]:
missing_coll_counts = (ps_collections
                       .filter(lower(col("collection")).isin(missing_collections))
                       .orderBy(col("count"), ascending=False)
                       )
print(missing_coll_counts.count())
missing_coll_counts.show(10)
(missing_coll_counts
 .select(sum(col("count")))
 .show()
)

53
+----------+-----+
|collection|count|
+----------+-----+
|       OSM|  491|
|      HUMZ|  396|
|       MCP|  251|
|     MBUCV|  190|
|       AMS|  102|
|    CAS-SU|   87|
|     MSNVR|   63|
|        SU|   62|
|       KIZ|   54|
|      MCNG|   49|
+----------+-----+
only showing top 10 rows

+----------+
|sum(count)|
+----------+
|      2038|
+----------+



### Joining 

In [27]:
ps_idb = (ps_spec
         .join(idb_df, 
               ((lower(col("collection")) == col("collectioncode")) 
                 | (lower(col("collection")) == col("institutioncode")))
               & (lower(col("catNum")) == col("catalognumber"))
              )
          )
print(ps_idb.count())
ps_idb.select(col("matrix_taxon"), col("collection"), col("catNum"), col("scientificname")).show(10)

22699
+--------------------+----------+------+--------------------+
|        matrix_taxon|collection|catNum|      scientificname|
+--------------------+----------+------+--------------------+
|Opisthopterus tar...|      USNM|283232|  bucephala clangula|
|    Etrumeus sardina|      USNM|188950|     etrumeus sadina|
|      Etrumeus teres|      USNM|188950|     etrumeus sadina|
|Pogonopoma werthe...|      USNM|302292|                null|
|     Brycinus brevis|      USNM|179332|     cancer borealis|
|Maculirhamdia sti...|      USNM|326389|myrmeciza atrotho...|
|Acrochordonichthy...|      FMNH| 68010|lasmigona costata...|
|  Galaxias fasciatus|      USNM|203882|vulpes vulpes ala...|
|     Notropis procne|        KU| 18877|      plesiosminthus|
|Glandulocauda mel...|      FMNH| 15026|lampsilis teres (...|
+--------------------+----------+------+--------------------+
only showing top 10 rows



It certainly looks like a bunch of the taxonomy does not match and there seem to be about double the number of expected records. There are likely multiple code/number pairs matching. A manual example made by looking in the iDigBio portal for institution code = "ku" and catalog number = "15595" yeilds 7 records found. Two examples:

http://portal.idigbio.org/portal/records/ed17b627-0d7d-4996-a9f2-6ae9bdb74ebb

http://portal.idigbio.org/portal/records/9c7f810f-6f42-403a-8adc-0d3b4866141e

Ignoring taxonomy, how much duplication is there?

In [29]:
(ps_idb
.groupBy(col("collection"), col("catNum"))
.count()
.orderBy(col("count"), ascending=False)
.show(20)
)

+----------+------+-----+
|collection|catNum|count|
+----------+------+-----+
|       MCZ|  7714|   25|
|      USNM|130183|   24|
|      USNM|288474|   18|
|      FMNH|  6724|   16|
|        KU| 18440|   16|
|        KU| 12442|   16|
|      FMNH| 10489|   16|
|      USNM|218830|   15|
|       CAS| 55554|   15|
|      USNM| 39529|   14|
|      USNM|232930|   14|
|      USNM|278989|   14|
|      FMNH|  7592|   14|
|      FMNH| 10064|   14|
|      USNM|220851|   12|
|      USNM|236399|   12|
|      USNM|273737|   12|
|      USNM|188950|   12|
|      USNM|229884|   12|
|      USNM|175436|   12|
+----------+------+-----+
only showing top 20 rows



Certainly a lot. What to add for taxonomy? Scientific name (or the concatenation of genus and specific epithet) in iDigBio and the matrix_taxon column have some variation with sp. and author junk. Would just matching the genus be good enough? Presumably that would disambiguate the multiple collections at an institution and address the apparent conflation of the instution code and collection code concepts that appear to be present in Phenoscape.

In [37]:
ps_idb_inc_tax = (ps_idb
                  .filter(lower(split(col("matrix_taxon")," ")[0]) == col("genus"))
                  )
print(ps_idb_inc_tax.count())
ps_idb_inc_tax.select(col("matrix_taxon"), col("collection"), col("catNum"), col("scientificname")).show(10)

2936
+--------------------+----------+------+--------------------+
|        matrix_taxon|collection|catNum|      scientificname|
+--------------------+----------+------+--------------------+
|    Etrumeus sardina|      USNM|188950|     etrumeus sadina|
|      Etrumeus teres|      USNM|188950|     etrumeus sadina|
|   Erimyzon succetta|      USNM|129386|    erimyzon sucetta|
|Argopleura magdal...|      USNM|235922|argopleura magdal...|
|      Argopleura sp.|      USNM|235922|argopleura magdal...|
|    Epapterus blohmi|      USNM|257983|    epapterus blohmi|
|Bunocephalus cora...|      USNM|284646|bunocephalus cora...|
|  Brycinus lateralis|      USNM|310101|  brycinus lateralis|
|  Eugnathichthys sp.|      USNM|326204|      eugnathichthys|
|Bathytyphlops mar...|      USNM|341861|bathytyphlops mar...|
+--------------------+----------+------+--------------------+
only showing top 10 rows



In [42]:
(ps_idb_inc_tax
.groupBy(col("genus"), col("collection"), col("catNum"))
.count()
.orderBy(col("count"), ascending=False)
.show(50)
)

+-----------------+----------+------+-----+
|            genus|collection|catNum|count|
+-----------------+----------+------+-----+
|     glyptothorax|      USNM|288474|    3|
|      astroblepus|      FMNH| 96628|    2|
|   breitensteinia|      USNM|230304|    2|
|     bunocephalus|        UF| 33074|    2|
|         etrumeus|      USNM|188950|    2|
|       citharinus|      USNM|231554|    2|
|          hoplias|      USNM|226265|    2|
|   aphyocheirodon|      FMNH| 57821|    2|
|      apteronotus|      FMNH|  7592|    2|
|      eigenmannia|      USNM|260240|    2|
|     bunocephalus|       MCZ| 48566|    2|
|     bunocephalus|       MCZ| 46133|    2|
|       argopleura|      ANSP|127516|    2|
|         brycinus|      USNM|310088|    2|
|      eigenmannia|      USNM|260242|    2|
|     bunocephalus|       ROM| 62223|    2|
|     bunocephalus|        UF| 77836|    2|
|           bagrus|      USNM|229884|    2|
|     bunocephalus|      USNM|121244|    2|
|      hypoptopoma|      FMNH| 8

Not perfect but better.

Write out the result to fiddle with more later.

In [44]:
ps_idb_inc_tax.write.mode('overwrite').parquet("/tmp/ps_idb_inc_tax.parquet")

Py4JJavaError: An error occurred while calling o893.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:215)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:65)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:173)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:145)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.ap

### Conclusions of Linking Using Specimen Identifiers

About 2900 specimen identifiers out of about 12000 valid ones (24%) were able to be linked to a single specimen in iDigBio using the catalog number, the genus, and by matching the collection code to either the institution or collection codes in iDigBio. There were 439 invalid catalog numbers that contained something other than a bare catalog number in Phenoscape. In addition, there were about 40 combinations of those three fields that occured more than once in iDigBio.

This match rate is pretty low. With some inspection there some international collections that are missing from iDigBio ("UFRJ" or Federal University of Rio de Janeiro, "HUMZ" or Hokkaido University, Laboratory of Marine Zoology, etc.) that account for 2038 unjoined records. An addtional (12283 - 2900 - 439 - 2038) / (12283) = 56% were unable to be matched to a single record with no explaination.

